In [8]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from datetime import datetime
# from main import getHourlyGen, getHourlyLoad

In [9]:
df = pd.read_excel('data/Refrence/fca_obligations.xlsx', skiprows=0, index_col=None, sheet_name='FCA 14')
df = df.drop(['Intermittent', 'DR Type', 'Active/ Passive', 'Interface ID', 'Interface Name', 'Dispatch Zone ID', 'Dispatch Zone Name',
              'Lead Participant ID', 'Lead Participant Name', 'De-list Bid Type', 'De-list Bid MW', 'Status', 'Capacity Zone ID', 'Capacity Zone Name', 'Load Zone'], axis=1)
new_columns = pd.to_datetime(df.columns[-12:]).strftime('%B')
df.columns = df.columns[:-12].tolist() + new_columns.tolist()
df = df[df['Type'] == 'Generator']
df.reset_index(drop=True, inplace=True)


In [10]:
from utilsData import getISO
from utilsData import fuelDict
dfISO, numGenerators, totalCap, totalCSO = getISO()

Total Capacity:  38389.70230400001 Number of Generators:  2585


In [11]:
# ?FIX THIS
def sina(x):
    # print(x)
    x = x.lower()
    if 'water' in x or 'hydro' in x:
        return 'Hydro'
    elif 'wind' in x:
        return 'Wind'
    elif 'waste' in x or 'biomass' in x or 'wood' in x:
        return 'Waste'
    elif 'solar' in x or 'photo' in x:
        return 'Solar'
    elif 'gas' in x or 'ker' in x:
        return 'Gas'
    elif 'coal' in x:
        return 'Coal'
    elif 'oil' in x or 'diesel' in x or 'jet' in x:
        return 'Oil'    
    elif 'nuclear' in x:
        return 'NUC'
    else:
        return 'Other'


In [17]:
# np.isin(df['ID'], dfISO['ASSET ID'])

In [12]:
fuels = df[['ID', 'Generating Fuel Type']]
fuels['Fuel Type'] = pd.Series()
for i in range(0, len(fuels['Fuel Type'])):
     a = dfISO[dfISO['ASSET ID'] == fuels.iloc[i]['ID']]['Fuel Type']
     if len(a) > 0:
          fuels['Fuel Type'][i] = a.to_list()[0]
     else:
          fuels['Fuel Type'][i] = sina(fuels['Generating Fuel Type'][i])

/tmp/ipykernel_19955/1635398450.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  fuels['Fuel Type'] = pd.Series()
/tmp/ipykernel_19955/1635398450.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuels['Fuel Type'] = pd.Series()
/tmp/ipykernel_19955/1635398450.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuels['Fuel Type'][i] = a.to_list()[0]
/tmp/ipykernel_19955/1635398450.py:8: SettingWithCopyWarning: 
A value is trying to be set on a

In [19]:
fuels = fuels.fillna('Other')
set(fuels['Fuel Type'])

{'Coal', 'Gas', 'Hydro', 'Nuclear', 'Oil', 'Other', 'Solar', 'Waste', 'Wind'}

In [20]:
df.insert(3, 'Fuel Type', fuels['Fuel Type'])
df.to_csv('data/CSO2023.csv', index=False)